## Challenge Description
You are looking at data from an e-commerce website. The site is very simple and has just 4 pages:
- The first page is the home page. When you come to the site for the first time, you can only land on the home page as a first page.
- From the home page, the user can perform a search and land on the search page.
- From the search page, if the user clicks on a product, she will get to the payment page, where she is asked to provide payment information in order to buy that product.
- If she does decide to buy, she ends up on the confirmation page

The company CEO isn't very happy with the volume of sales and, especially, of sales coming from new users. Therefore, she asked you to investigate whether there is something wrong in the conversion funnel or, in general, if you could suggest how conversion rate can be improved.
Specifically, she is interested in :
- A full picture of funnel conversion rate for both desktop and mobile
- Some insights on what the product team should focus on in order to improve conversion rate as well as anything you might discover that could help improve conversion rate.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import bisect
import collections
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn import tree
import graphviz

sys.path.append('/Users/pinecone/Git/TakeHomePractice')
import func

/Users/pinecone/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
!ls data/

home_page_table.csv            search_page_table.csv
payment_confirmation_table.csv user_table.csv
payment_page_table.csv


In [3]:
home_page = pd.read_csv('data/home_page_table.csv')
search_page = pd.read_csv('data/search_page_table.csv')
payment_confirmation = pd.read_csv('data/payment_confirmation_table.csv')
user = pd.read_csv('data/user_table.csv')
payment_page = pd.read_csv('data/payment_page_table.csv')


In [4]:
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [5]:
search_page.head()

,user_id,page
0,15866,search_page
1,347058,search_page
2,577020,search_page
3,780347,search_page
4,383739,search_page


In [6]:
payment_confirmation.head()

,user_id,page
0,123100,payment_confirmation_page
1,704999,payment_confirmation_page
2,407188,payment_confirmation_page
3,538348,payment_confirmation_page
4,841681,payment_confirmation_page


In [7]:
user.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [8]:
payment_page.head()

,user_id,page
0,253019,payment_page
1,310478,payment_page
2,304081,payment_page
3,901286,payment_page
4,195052,payment_page


No missing data in all files!

In [10]:
print(func.missing_data(home_page))
print(func.missing_data(search_page))
print(func.missing_data(payment_confirmation))
print(func.missing_data(user))
print(func.missing_data(payment_page))

        user_id    page
Total         0       0
Percent       0       0
Types     int64  object
        user_id    page
Total         0       0
Percent       0       0
Types     int64  object
        user_id    page
Total         0       0
Percent       0       0
Types     int64  object
        user_id    date  device     sex
Total         0       0       0       0
Percent       0       0       0       0
Types     int64  object  object  object
        user_id    page
Total         0       0
Percent       0       0
Types     int64  object


Now, let's combine these tables.

In [12]:
data = pd.merge(left=user, right=home_page, how='left', on='user_id')
data = pd.merge(left=data, right=search_page, how='left', on='user_id', suffixes=('_home', '_search'))
data = pd.merge(left=data, right=payment_page, how='left', on='user_id')
data = pd.merge(left=data, right=payment_confirmation, how='left', on='user_id', suffixes=('_payment', '_confirmation'))
data.head()

,user_id,date,device,sex,page_home,page_search,page_payment,page_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90400 entries, 0 to 90399
Data columns (total 8 columns):
user_id              90400 non-null int64
date                 90400 non-null object
device               90400 non-null object
sex                  90400 non-null object
page_home            90400 non-null object
page_search          45200 non-null object
page_payment         6030 non-null object
page_confirmation    452 non-null object
dtypes: int64(1), object(7)
memory usage: 6.2+ MB


In [14]:
func.missing_data(data)

,user_id,date,device,sex,page_home,page_search,page_payment,page_confirmation
Total,0,0,0,0,0,45200,84370,89948
Percent,0,0,0,0,0,50,93.3296,99.5
Types,int64,object,object,object,object,object,object,object


## Analysis
Let's check the visit probability for each page. We can use the missing_data function to get it.

In [34]:
funnel = func.missing_data(data.iloc[:, [4, 5, 6, 7]]).iloc[[0, 1], :].T
funnel['Conversion%'] = 100 - funnel.Percent

In [37]:
print(funnel.iloc[:, [2]])

                  Conversion%
page_home                 100
page_search                50
page_payment          6.67035
page_confirmation         0.5


We can see that the visit probability decreases very fast.

## Conversion rate
Now let's compute the conversion rate.

In [38]:
def conversion_rate(df):
    rates = []
    tmp = df[~df['page_home'].isnull()]
    rates.append(1 - tmp['page_search'].isnull().sum() / len(tmp))
    
    tmp = df[~df['page_search'].isnull()]
    rates.append(1 - tmp['page_payment'].isnull().sum() / len(tmp))
    
    tmp = df[~df['page_payment'].isnull()]
    rates.append(1 - tmp['page_confirmation'].isnull().sum() / len(tmp))
    
    return rates

1. overall conversion rate

In [43]:
rates = conversion_rate(data)
maps = {"Page": ['home', 'search', 'payment'], 'Conversion Rate': rates}
overall_conversion = pd.DataFrame(maps, columns=['Page', 'Conversion Rate'])
overall_conversion

,Page,Conversion Rate
0,home,0.500000
1,search,0.133407
2,payment,0.074959


2. device

In [49]:
desk_rates = conversion_rate(data.loc[data.device == 'Desktop', :])
mobile_rates = conversion_rate(data.loc[data.device == 'Mobile', :])
maps = {"Page": ['home', 'search', 'payment'], 'desk_CR': desk_rates}
device_conversion = pd.DataFrame(maps, columns=['Page', 'desk_CR'])
device_conversion['mobile_CR'] = mobile_rates
device_conversion

,Page,desk_CR,mobile_CR
0,home,0.500000,0.5
1,search,0.100000,0.2
2,payment,0.049834,0.1


3. sex

In [ ]:
male_rates = conversion_rate(data.loc[data.sex == 'Male', :])
female_rates = conversion_rate(data.loc[data.sex == 'Female', :])
maps = {"Page": ['home', 'search', 'payment'], 'male_CR': male_rates}
sex_conversion = pd.DataFrame(maps, columns=['Page', 'male_CR'])
device_conversion['mobile_CR'] = mobile_rates
device_conversion